In [1]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Carga tu archivo .env
load_dotenv()

# --- Forzamos las variables de entorno ---
# 1. Clave de OpenAI
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
# 2. URL de OpenAI (Evita el error 'UnsupportedProtocol')
os.environ["OPENAI_API_BASE_URL"] = "https://api.openai.com/v1"

# Configuración del LLM
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        openai_api_base=os.environ.get("OPENAI_API_BASE_URL"),
        openai_api_key=os.environ.get("OPENAI_API_KEY"),
        temperature=0
    )
    print("✅ LLM de LangChain configurado.")
except Exception as e:
    print(f"❌ Error configurando el LLM: {e}")
    llm = None

✅ LLM de LangChain configurado.


In [2]:
# Tuvimos que instalar 'wikipedia' y 'duckduckgo-search' por separado
from crewai.tools import BaseTool

# --- Herramienta de Consulta (DuckDuckGo) ---
# Importamos la herramienta oficial de LangChain que CrewAI puede usar
from langchain_community.tools import DuckDuckGoSearchRun
# La "envolvemos" para que CrewAI la entienda
class DuckDuckGoTool(BaseTool):
    name: str = "Búsqueda Web (DuckDuckGo)"
    description: str = "Busca en la web tiendas, recomendaciones o precios. Útil para información dinámica."
    
    def _run(self, query: str) -> str:
        searcher = DuckDuckGoSearchRun()
        return searcher.run(query)

# --- Herramienta de Consulta (Wikipedia) ---
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
class WikipediaTool(BaseTool):
    name: str = "Búsqueda en Wikipedia"
    description: str = "Busca en Wikipedia definiciones o hechos históricos. Útil para información fáctica."
    
    def _run(self, query: str) -> str:
        api = WikipediaAPIWrapper(lang="es", top_k_results=1, doc_content_chars_max=2000)
        searcher = WikipediaQueryRun(api_wrapper=api)
        return searcher.run(query)

# --- Herramienta de Escritura ---
# Esta es una herramienta personalizada simple
@BaseTool.tool("Escritor de Archivos")
def file_write_tool(filename: str, content: str) -> str:
    """Escribe el 'content' (contenido) de texto dado en un archivo con el 'filename' (nombre de archivo) especificado."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"Éxito: Contenido guardado en '{filename}'."
    except Exception as e:
        return f"Error al escribir archivo: {e}"

# Lista de todas las herramientas
tools = [DuckDuckGoTool(), WikipediaTool(), file_write_tool]
print("✅ Herramientas de Consulta (DuckDuckGo, Wiki) y Escritura (Archivos) listas.")

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
from crewai import Agent

piercing_expert_agent = Agent(
    role='Asistente Experto en Modificación Corporal',
    
    goal="""Tu objetivo es ser un asistente integral sobre piercings. Debes:
    1. Conversar amigablemente con el usuario y **recordar el historial (Memoria)**.
    2. Usar la 'Búsqueda Web (DuckDuckGo)' para buscar **tiendas, recomendaciones o precios**.
    3. Usar 'Búsqueda en Wikipedia' para buscar **definiciones o tipos de piercings**.
    4. Usar el 'Escritor de Archivos' para **guardar resúmenes** cuando el usuario lo pida.
    5. **Planificar la secuencia** de herramientas a usar si una tarea es compleja (ej. "busca esto Y guárdalo").""",
    
    backstory="Eres un experto de renombre mundial en piercings y tatuajes. Tienes un profundo conocimiento de seguridad, cuidados, y tendencias.",
    tools=tools,
    llm=llm,
    memory=True, # Habilita la memoria en el agente (IL2.2)
    verbose=False,
    allow_delegation=False
)
print("✅ Agente 'Experto' creado con Memoria, Planificación y Herramientas.")

In [ ]:
from crewai import Task, Crew

expert_task = Task(
    description="Analiza la solicitud del usuario: '{query}'. "
                "Usa tu razonamiento para decidir qué herramientas necesitas. "
                "Si es una charla, usa tu memoria. "
                "Si es una búsqueda, usa Google (DuckDuckGo) o Wikipedia. "
                "Si es una orden de escritura, usa la herramienta de archivos.",
    expected_output="Una respuesta completa y útil a la solicitud del usuario.",
    agent=piercing_expert_agent
)

piercing_crew = Crew(
    agents=[piercing_expert_agent],
    tasks=[expert_task],
    memory=True, # Habilita la memoria en el crew (IL2.2)
    verbose=False
)
print("✅ Crew listo con Agente Experto y Tarea genérica.")

In [ ]:
# --- PRUEBA 1 (DuckDuckGo) ---
query_1 = "Necesito recomendaciones de tiendas de piercing seguras en Santiago de Chile"
response_1 = piercing_crew.kickoff(inputs={'query': query_1})
print(f"--- 💬 CONSULTA 1: {query_1} ---")
print(f"--- 🤖 RESPUESTA 1: {response_1} ---")

In [ ]:
# --- PRUEBA 2 (Memoria) ---
query_2 = "¿Sobre qué ciudad te acabo de preguntar?"
response_2 = piercing_crew.kickoff(inputs={'query': query_2})
print(f"--- 💬 CONSULTA 2: {query_2} ---")
print(f"--- 🤖 RESPUESTA 2: {response_2} ---")

In [ ]:
# --- PRUEBA 3 (Wikipedia) ---
query_3 = "¿Qué es un piercing 'helix' según Wikipedia?"
response_3 = piercing_crew.kickoff(inputs={'query': query_3})
print(f"--- 💬 CONSULTA 3: {query_3} ---")
print(f"--- 🤖 RESPUESTA 3: {response_3} ---")

In [ ]:
# --- PRUEBA 4 (Planificación: Buscar + Escribir) ---
query_4 = "Busca en Wikipedia los cuidados para un piercing helix y guarda esa información en el archivo 'cuidados.txt'."
response_4 = piercing_crew.kickoff(inputs={'query': query_4})
print(f"--- 💬 CONSULTA 4: {query_4} ---")
print(f"--- 🤖 RESPUESTA 4: {response_4} ---")
# (Revisa tu carpeta, debería aparecer 'cuidados.txt')